In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def histogram_equalization_v_channel(image):
    # Convert image from BGR to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Compute histogram of V channel
    hist, bins = np.histogram(v.flatten(), 256, [0,256])
    
    # Compute the cumulative distribution function (CDF)
    cdf = hist.cumsum()
    
    # Normalize the CDF (avoid division by zero)
    cdf_masked = np.ma.masked_equal(cdf, 0)
    cdf_masked = (cdf_masked - cdf_masked.min()) * 255 / (cdf_masked.max() - cdf_masked.min())
    cdf_final = np.ma.filled(cdf_masked, 0).astype('uint8')
    
    # Map the V channel values using the equalized CDF
    v_equalized = cdf_final[v]
    
    # Merge the channels back and convert to BGR color space
    hsv_equalized = cv2.merge([h, s, v_equalized])
    result = cv2.cvtColor(hsv_equalized, cv2.COLOR_HSV2BGR)
    return result, hsv

In [9]:
def plot_and_save_2d_histogram(hsv, output_filename):
    # Extract H and S channels
    h_channel = hsv[:, :, 0]
    s_channel = hsv[:, :, 1]
    
    # Compute a 2D histogram with 50 bins for each channel.
    hist_2d, x_edges, y_edges = np.histogram2d(
        h_channel.flatten(), s_channel.flatten(), bins=50, range=[[0,180], [0,256]]
    )
    
    plt.figure(figsize=(6, 5))
    plt.imshow(hist_2d, interpolation='nearest', origin='lower', aspect='auto')
    plt.title('2D Histogram of H and S channels')
    plt.xlabel('Hue bins')
    plt.ylabel('Saturation bins')
    plt.colorbar()
    plt.savefig(output_filename)
    plt.close()


In [10]:
def plot_pdf_comparison(original, equalized, output_filename):
    # Convert both images to grayscale
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    equalized_gray = cv2.cvtColor(equalized, cv2.COLOR_BGR2GRAY)
    
    # Compute the intensity histograms (PDFs)
    hist_orig = cv2.calcHist([original_gray], [0], None, [256], [0, 256])
    hist_eq = cv2.calcHist([equalized_gray], [0], None, [256], [0, 256])
    
    plt.figure(figsize=(10, 5))
    plt.plot(hist_orig, label='Original')
    plt.plot(hist_eq, label='Equalized')
    plt.title('Grayscale PDF Comparison')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.legend()
    plt.savefig(output_filename)
    plt.close()



In [11]:
def create_result_comparison_image(original, equalized, output_filename):
    # Create a side-by-side comparison image
    comparison = np.hstack((original, equalized))
    cv2.imwrite(output_filename, comparison)



In [ ]:
def main():
    # Read the input image. Make sure 'input.png' exists in the working directory.
    image = cv2.imread('input.png')
    if image is None:
        print("Error: 'input.png' not found!")
        return

    # Perform histogram equalization on the V channel.
    equalized_image, hsv_original = histogram_equalization_v_channel(image)
    
    # Save the equalized image as 'output.png'
    cv2.imwrite('output.png', equalized_image)
    
    # Generate and save the 2D histogram of H and S channels.
    plot_and_save_2d_histogram(hsv_original, '2d_histogram.png')
    
    # Plot and save the grayscale PDF comparison for the original and equalized images.
    plot_pdf_comparison(image, equalized_image, 'pdf_comparison.png')
    
    # Create and save the side-by-side comparison image.
    create_result_comparison_image(image, equalized_image, 'result.png')
    
    print("All images have been saved successfully.")



In [13]:
if __name__ == "__main__":
    main()

All images have been saved successfully.
